In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5"


In [ ]:
import pickle
from typing import List, Tuple, Union

import torch
from torch import nn, Tensor
from torch.utils.data import DataLoader, random_split
from torch.nn.utils.rnn import pack_sequence, PackedSequence

from datautils import SeqDataset
from models import BiGRUClassifier
from trainutils import device, train


# Hyperparameter


In [ ]:
batch_size = 16
learning_rate = 1e-4
trial_name = "Bi-GRU"


# Define Dataset && DataLoader


In [ ]:
with open("./zscore.skl", "rb") as fp:
    scaler = pickle.load(fp)

train_dataset = SeqDataset("./Dataset", scaler)
num_speakers = train_dataset.speaker_count()

train_len = int(len(train_dataset) * 0.8)
valid_len = len(train_dataset) - train_len
train_dataset, valid_dataset = random_split(train_dataset, [train_len, valid_len])


In [ ]:
def collate(
    batch: Union[List[Tuple[Tensor, Tensor]], List[Tensor]]
) -> Union[Tuple[PackedSequence, Tensor], List[PackedSequence]]:
    if isinstance(batch[0], tuple):
        features, labels = zip(*batch)
        return pack_sequence(features, enforce_sorted=False), torch.cat(labels)
    else:
        return pack_sequence(batch, enforce_sorted=False)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True, collate_fn=collate)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True, collate_fn=collate)


# Define Model


In [ ]:
model = BiGRUClassifier(embed_size=40, hidden_size=256, num_layers=2, num_classes=num_speakers).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [ ]:
train(train_dataloader, valid_dataloader, model, criterion, optimizer, 5000, 100, trial_name)
